# Quick Dash (dashit)

```python
@deepgreen.api
def magic():
    return 1
```

Spins up a server `if __name__ == "__main__"` else 

## My Function

In [5]:
def magic(x,y,z, option1="nothing", option2="another", **kwargs):
    """My Function Does Everything
    
    First, it sets it up.
    
    Then, it knocks it down.
    
    """
    i,j,k = kwargs.pop("i",""), kwargs.pop("j",""), kwargs.pop("k","")
    return f"{x},{y},{z} -> magic <- {option1} {option2} ||| {i},{j},{k}"

In [7]:
import inspect
inspect.getfullargspec(magic)

FullArgSpec(args=['x', 'y', 'z', 'option1', 'option2'], varargs=None, varkw='kwargs', defaults=('nothing', 'another'), kwonlyargs=[], kwonlydefaults=None, annotations={})

# TODO: 

* named args should be expected as URL params .. that just makes sense

In [158]:
from typing import List, Callable

from functools import partial
import inspect
import pandas as pd

import flask
    
import dash
import dash_html_components as html
import dash_core_components as dcc

def dashit(functions: List[Callable], appname: str):
    """
    Create a QUICK Dash app exposing your functions as API endpoints!
    
    endpoint url:  
    
    All positional and named arguments will be required in the url
    """
    app = dash.Dash(__name__, url_base_pathname = f"/{appname}/")

    def inject_flask_params_as_kwargs(func, **kwargs):
        """Flask converts the url variables into kwargs. We pass those back to our function as args. We also want to inject as kwargs the optional URL parameters"""
        args = kwargs
        kwargs = flask.request.args.to_dict(flat=False)
        print(args, kwargs)
        return func(**args, **kwargs )

    def handle_wacky_types(thing):
        if isinstance(thing, pd.DataFrame):
            thing = thing._repr_html_()
        return thing
    
    def all_the_small_things(func, **kwargs):
        response = inject_flask_params_as_kwargs(func, **kwargs)
        response = handle_wacky_types(response)
        return response
    
    def add_rule(app, f):
        """
        Register function as Flask route. Positional and named arguments are all required. 
        In order to handle optional arguments, use **kwargs. *args, is right out.
        """

        APP_BASE = app.config["url_base_pathname"]

        argspec = inspect.getfullargspec(f)
        argspec

        BASE = f.__name__
        args = argspec.args
        args = "".join([f"/<{arg}>" for arg in argspec.args])

        rule = f"{APP_BASE}{BASE}{args}"
        app.server.add_url_rule(rule, endpoint = f.__name__, view_func = partial(all_the_small_things,func=f))
        return rule
    
    new_routes = [{"name": func.__name__, "docstring": inspect.cleandoc(func.__doc__) if func.__doc__ else "", "endpoint": add_rule(app,func)} for func in functions]

    def generate_endpoint_html(route):
        return dcc.Markdown([f"""**{route["name"]}:** [{route["endpoint"]}]({route["endpoint"]})"""]), html.Blockquote([dcc.Markdown([route["docstring"]])])
    flatten = lambda l: [item for sublist in l for item in sublist]

    app.layout = html.Div([
        html.H1([appname]),
        html.H3(["DASHIT: Functions Are APIs Now."]), #some explanation on how the app works. Use function docstring.
        html.Div(flatten([
            generate_endpoint_html(route) for route in new_routes
        ]))
    ])
    print(new_routes)
    
    return app

In [159]:
app = dashit([magic], "Titanic")

[{'name': 'magic', 'docstring': 'My Function Does Everything\n\nFirst, it sets it up.\n\nThen, it knocks it down.', 'endpoint': '/Titanic/magic/<x>/<y>/<z>/<option1>/<option2>'}]


In [102]:
app.run_server(host="0.0.0.0", port=16900)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:16900/ (Press CTRL+C to quit)
49.37.49.68 - - [24/Mar/2020 23:50:45] "GET /Titanic/ HTTP/1.1" 200 -
49.37.49.68 - - [24/Mar/2020 23:50:45] "GET /Titanic/_dash-component-suites/dash_renderer/polyfill@7.v1_2_3m1584560928.7.0.min.js HTTP/1.1" 200 -
49.37.49.68 - - [24/Mar/2020 23:50:46] "GET /Titanic/_dash-component-suites/dash_renderer/react@16.v1_2_3m1584560928.8.6.min.js HTTP/1.1" 200 -
49.37.49.68 - - [24/Mar/2020 23:50:46] "GET /Titanic/_dash-component-suites/dash_renderer/react-dom@16.v1_2_3m1584560928.8.6.min.js HTTP/1.1" 200 -
49.37.49.68 - - [24/Mar/2020 23:50:46] "GET /Titanic/_dash-component-suites/dash_renderer/prop-types@15.v1_2_3m1584560928.7.2.min.js HTTP/1.1" 200 -
49.37.49.68 - - [24/Mar/2020 23:50:46] "GET /Titanic/_dash-component-suites/dash_core_components/dash_core_components.v1_7_0m1584560928.min.js HTTP/1.1" 200 -
49.37.49.68 - - [24/Mar/2020 23:50:46] "GET /Titanic/_dash-component-suites/dash_core_components/dash_core_components-shared.

# Spin up an api

From:

* an ML model
* a function

In [160]:
def titanic_drill(**kwargs):
    fields = kwargs.pop("fields", [])
    filters = kwargs.pop("filters", "")
    df = deepgreen.drill(fields=fields, filters=filters, path="/data/deepgreen/user/duranl4/myfirstdata")
    return df

In [161]:
def titanic_group(**kwargs):
    fields = kwargs.pop("fields", [])
    filters = kwargs.pop("filters", "")
    aggFunc = kwargs.pop("aggFunc", {})
    df = deepgreen.group(aggFunc=aggFunc, fields=fields, filters=filters, path="/data/deepgreen/user/duranl4/myfirstdata")
    return df

In [162]:
app = dashit([titanic_drill, titanic_group], "Titanic")

[{'name': 'titanic_drill', 'docstring': '', 'endpoint': '/Titanic/titanic_drill'}, {'name': 'titanic_group', 'docstring': '', 'endpoint': '/Titanic/titanic_group'}]


http://notebook.tds.td.com:16900/Titanic/titanic_drill?filters=Cabin=B58%20B60&fields=Survived&fields=PClass&fields=Name

In [164]:
app.run_server(host="0.0.0.0", port=16900)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:16900/ (Press CTRL+C to quit)
49.37.49.68 - - [25/Mar/2020 00:10:29] "GET /Titanic/ HTTP/1.1" 200 -
49.37.49.68 - - [25/Mar/2020 00:10:29] "GET /Titanic/_dash-layout HTTP/1.1" 200 -
49.37.49.68 - - [25/Mar/2020 00:10:29] "GET /Titanic/_dash-dependencies HTTP/1.1" 200 -
49.37.49.68 - - [25/Mar/2020 00:10:29] "GET /Titanic/_favicon.ico?v=1.8.0 HTTP/1.1" 200 -
49.37.49.68 - - [25/Mar/2020 00:10:42] "GET /Titanic/titanic_drill?filters=Cabin=B58%20B60&fields=Survived&fields=PClass&fields=Name HTTP/1.1" 200 -


{} {'filters': ['Cabin=B58 B60'], 'fields': ['Survived', 'PClass', 'Name']}


49.37.49.68 - - [25/Mar/2020 00:11:58] "GET /Titanic/titanic_drill?filters=Cabin=B58%20B60&fields=Survived&fields=PClass&fields=Name HTTP/1.1" 200 -


{} {'filters': ['Cabin=B58 B60'], 'fields': ['Survived', 'PClass', 'Name']}


# Next

* Named arguments (optional) should be coerced into parameters
* Maybe a decorator?